In [1]:
# STEP 8: Model Validation
# ------------------------------------------------

import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve

# --- Load Data ---
file_path = r"/content/Customer_Final_Scores_with_Calibration.xlsx"
df_raw = pd.read_excel(file_path, sheet_name="RawData")
df_decile = pd.read_excel(file_path, sheet_name="DecileCalibration")

# Target and Score
y_true = df_raw["Default_y"]
# Flip the score so higher score = better customer
y_score = -df_raw["Final_Score"]

In [2]:
# 1. Gini & AR
# ------------------------------------------------
auc = roc_auc_score(y_true, y_score)
gini = 2 * auc - 1
ar = gini / 2

print(f"Gini: {gini:.3f}")
print(f"Accuracy Ratio (AR): {ar:.3f}")

Gini: 0.761
Accuracy Ratio (AR): 0.380


In [3]:
# 2. KS Statistic
# ------------------------------------------------
fpr, tpr, thresholds = roc_curve(y_true, y_score)
ks_stat = max(tpr - fpr)
print(f"KS Statistic: {ks_stat:.3f}")

KS Statistic: 0.686


In [4]:
# 3. Rank Order
# ------------------------------------------------
df_raw["score_decile_10"] = pd.qcut(y_score, 10, labels=False, duplicates="drop")

rank_order = df_raw.groupby("score_decile_10").agg(
    total=("Default_y", "count"),
    bads=("Default_y", "sum"),
    goods=("Default_y", lambda x: (x == 0).sum()),
    bad_rate=("Default_y", "mean"),
    avg_score=("Final_Score", "mean")
).reset_index()

print("\nRank Order Table:")
print(rank_order)


Rank Order Table:
   score_decile_10  total  bads  goods  bad_rate   avg_score
0                0     10     1      9  0.100000  960.900000
1                1     10     0     10  0.000000  880.300000
2                2     10     0     10  0.000000  843.500000
3                3     10     1      9  0.100000  801.900000
4                4     10     2      8  0.200000  749.900000
5                5     10     1      9  0.100000  694.700000
6                6     11     4      7  0.363636  608.454545
7                7      9     5      4  0.555556  515.222222
8                8     10     8      2  0.800000  410.900000
9                9     10     8      2  0.800000  219.300000


In [5]:
# 4. PSI (Population Stability Index)
# ------------------------------------------------
def calculate_psi(expected, actual, buckets=10):
    breakpoints = np.arange(0, buckets + 1) / buckets * 100
    breakpoints = np.percentile(expected, breakpoints)

    expected_percents = np.histogram(expected, breakpoints)[0] / len(expected)
    actual_percents = np.histogram(actual, breakpoints)[0] / len(actual)

    psi_value = np.sum((actual_percents - expected_percents) *
                       np.log((actual_percents + 1e-6) / (expected_percents + 1e-6)))
    return psi_value

psi_val = calculate_psi(df_raw["Final_Score"], df_decile["calibrated_score"])
print(f"\nPSI: {psi_val:.3f}")


PSI: 10.054


In [6]:
# 5. Score Concentration
# ------------------------------------------------
score_threshold = np.percentile(df_raw["Final_Score"], 80)
concentration = (df_raw["Final_Score"] >= score_threshold).mean()
print(f"Score Concentration (Top 20% customers): {concentration:.2%}")


Score Concentration (Top 20% customers): 20.00%


In [7]:
# 6. Concordance / Discordance
# ------------------------------------------------
def concordance_discordance(y_true, y_score):
    good_scores = y_score[y_true == 0]
    bad_scores = y_score[y_true == 1]
    total_pairs = len(good_scores) * len(bad_scores)

    concordant = discordant = ties = 0
    for g in good_scores:
        concordant += np.sum(g > bad_scores)
        discordant += np.sum(g < bad_scores)
        ties += np.sum(g == bad_scores)

    return {
        "concordant_pct": concordant / total_pairs,
        "discordant_pct": discordant / total_pairs,
        "ties_pct": ties / total_pairs,
    }

conc_disc = concordance_discordance(y_true.values, y_score.values)
print("\nConcordance / Discordance:")
print(conc_disc)


Concordance / Discordance:
{'concordant_pct': np.float64(0.11952380952380952), 'discordant_pct': np.float64(0.8804761904761905), 'ties_pct': np.float64(0.0)}
